In [1]:
import pandas as pd

In [2]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
    border-style: solid;
}

In [3]:
school_file_path = "Resources/schools_complete.csv"
student_file_path = "Resources/students_complete.csv"
school_df = pd.read_csv(school_file_path)
school_df

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [4]:
student_df = pd.read_csv(student_file_path)
student_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
# school_df.count()
# school_df.dtypes
# student_df.count()
# student_df.dtypes

In [6]:
# District Summary
total_schools = school_df["School ID"].count()
total_students = student_df["Student ID"].count()
total_budget = school_df["budget"].sum()
avg_math_score = student_df["math_score"].mean()
avg_reading_score = student_df["reading_score"].mean()
high_math_score_df = student_df.loc[student_df["math_score"]>=70]
high_math_score_df.head()
stu_perc_high_mathscore = (high_math_score_df["Student ID"].count()/total_students)*100
high_reading_score_df = student_df.loc[student_df["reading_score"]>=70]
high_reading_score_df.head()
high_reading_score_df.count()
stu_perc_high_readingscore = (high_reading_score_df["Student ID"].count()/total_students)*100
overall_average_score = (avg_math_score + avg_reading_score)/2
dist_summary_df = pd.DataFrame({"Total Schools": [total_schools],"Total Students":[total_students], 
                               "Total bugdet": [total_budget], "Average Math Score": [avg_math_score], 
                               "Average Reading Score": [avg_reading_score], "% Passing Math": [stu_perc_high_mathscore],
                              "% Passing Reading": [stu_perc_high_readingscore], "Overall Average Score": [overall_average_score]})
dist_summary_df["Total Students"] = dist_summary_df["Total Students"].map("{:,}".format)
dist_summary_df["Total bugdet"] = dist_summary_df["Total bugdet"].map("${:,.2f}".format)
dist_summary_df

,Total Schools,Total Students,Total bugdet,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Average Score
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


In [7]:
# Create school data group by school name
school_group_df = school_df.groupby(["school_name", "type"]).agg({"budget":"sum", "size":"sum"})
school_group_df = school_group_df.reset_index(1)
school_group_df = school_group_df.rename(columns={"type": "School Type", "budget": "Total School Budget", "size":"School Size"})
school_group_df

# get the number of student, who got math_score>=70
def high_math_score_stu(values):
    math_stu_list = [
        1
        for x in values
        if x>=70
    ]
    return len(math_stu_list)

# get the number of student, who got reading_score>=70
def high_reading_score_stu(values):
    reading_stu_list = [
        1
        for x in values
        if x>=70
    ]
    return len(reading_stu_list)

# Create Student data group by schhol name
student_group_df = student_df.groupby("school_name").agg({"Student ID": "count", "math_score":["mean",high_math_score_stu],
                                                                   "reading_score":["mean", high_reading_score_stu]})
student_group_df.columns = student_group_df.columns.map("_".join)
student_group_df = student_group_df.rename(columns={"Student ID_count":"Total Students", "math_score_mean": "Average Math Score",
                                                     "math_score_high_math_score_stu": "High Math Score Stu", "reading_score_mean":"Average Reading Score", 
                                                      "reading_score_high_reading_score_stu" : "High Reading Score Stu"})
student_group_df
# Calculate "% Passing Math", "% Passing Reading", "% Overall Passing", and add the column respectively to "student_group_df".
student_group_df["% Passing Math"] = (student_group_df["High Math Score Stu"]/student_group_df["Total Students"])*100
student_group_df["% Passing Reading"] = (student_group_df["High Reading Score Stu"]/student_group_df["Total Students"])*100
student_group_df["% Overall Passing Rate"] = (student_group_df["% Passing Math"]+student_group_df["% Passing Reading"])/2
# drop unnecessary columns
student_group_df = student_group_df.drop(["High Math Score Stu", "High Reading Score Stu"], axis = 1)
# merge school data and student data and calculate "Per Student Budget".
school_student_summary_df = pd.merge(school_group_df[["Total School Budget", "School Type"]], student_group_df, how="inner", on="school_name")
school_student_summary_df["Per Student Budget"] = school_student_summary_df["Total School Budget"]/ school_student_summary_df["Total Students"]
# Rearrange the columns
school_student_summary_df = school_student_summary_df[["School Type", "Total Students", "Total School Budget",
                                                      "Per Student Budget", "Average Math Score", "Average Reading Score",
                                                       "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
school_student_summary_df["Total School Budget"] = school_student_summary_df["Total School Budget"].map("${:,.2f}".format)
school_student_summary_df["Per Student Budget"] = school_student_summary_df["Per Student Budget"].map("${:.2f}".format)
school_student_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668


In [8]:
# Get the first 5 top performing school list
high_performing_school_df = school_student_summary_df.sort_values(by="% Overall Passing Rate", ascending = False)
high_performing_school_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [9]:
# Get bottom 5 school list performance wise
bottom_performing_df = high_performing_school_df[-5:]
bottom_performing_df = bottom_performing_df.sort_values("% Overall Passing Rate",ascending=True)
bottom_performing_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [10]:
# Math score by grade
stud_9thgrade_df = student_df.loc[student_df["grade"]=="9th"]
stud_9thgrade_math_df = stud_9thgrade_df.groupby("school_name").agg({"math_score":"mean"})
stud_9thgrade_math_df = stud_9thgrade_math_df.rename(columns={"math_score":"9th"})
stud_9thgrade_math_df
stud_10thgrade_df = student_df.loc[student_df["grade"]=="10th"]
stud_10thgrade_math_df = stud_10thgrade_df.groupby("school_name").agg({"math_score":"mean"})
stud_10thgrade_math_df = stud_10thgrade_math_df.rename(columns={"math_score":"10th"})

stud_11thgrade_df = student_df.loc[student_df["grade"]=="11th"]
stud_11thgrade_math_df = stud_11thgrade_df.groupby("school_name").agg({"math_score":"mean"})
stud_11thgrade_math_df = stud_11thgrade_math_df.rename(columns={"math_score":"11th"})

stud_12thgrade_df = student_df.loc[student_df["grade"]=="12th"]
stud_12thgrade_math_df = stud_12thgrade_df.groupby("school_name").agg({"math_score":"mean"})
stud_12thgrade_math_df = stud_12thgrade_math_df.rename(columns={"math_score":"12th"})
stud_math_score_gradelevel_df = pd.merge(stud_9thgrade_math_df, stud_10thgrade_math_df, how="inner", on="school_name")
stud_math_score_gradelevel_df = pd.merge(stud_math_score_gradelevel_df, stud_11thgrade_math_df, how="inner", on="school_name")
stud_math_score_gradelevel_df = pd.merge(stud_math_score_gradelevel_df, stud_12thgrade_math_df, how="inner", on="school_name")
stud_math_score_gradelevel_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [11]:
# Reading score by grade
stud_9thgrade_df = student_df.loc[student_df["grade"]=="9th"]
stud_9thgrade_read_df = stud_9thgrade_df.groupby("school_name").agg({"reading_score":"mean"})
stud_9thgrade_read_df = stud_9thgrade_read_df.rename(columns={"reading_score":"9th"})

stud_10thgrade_df = student_df.loc[student_df["grade"]=="10th"]
stud_10thgrade_read_df = stud_10thgrade_df.groupby("school_name").agg({"reading_score":"mean"})
stud_10thgrade_read_df = stud_10thgrade_read_df.rename(columns={"reading_score":"10th"})

stud_11thgrade_df = student_df.loc[student_df["grade"]=="11th"]
stud_11thgrade_read_df = stud_11thgrade_df.groupby("school_name").agg({"reading_score":"mean"})
stud_11thgrade_read_df = stud_11thgrade_read_df.rename(columns={"reading_score":"11th"})

stud_12thgrade_df = student_df.loc[student_df["grade"]=="12th"]
stud_12thgrade_read_df = stud_12thgrade_df.groupby("school_name").agg({"reading_score":"mean"})
stud_12thgrade_read_df = stud_12thgrade_read_df.rename(columns={"reading_score":"12th"})

stud_reading_score_gradelevel_df = pd.merge(stud_9thgrade_read_df, stud_10thgrade_read_df, how="inner", on="school_name")
stud_reading_score_gradelevel_df = pd.merge(stud_reading_score_gradelevel_df, stud_11thgrade_read_df, how="inner", on="school_name")
stud_reading_score_gradelevel_df = pd.merge(stud_reading_score_gradelevel_df, stud_12thgrade_read_df, how="inner", on="school_name")
stud_reading_score_gradelevel_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [12]:
# scores by school spending 
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_student_summary_df["Per Student Budget"] = school_student_summary_df["Per Student Budget"].replace({"\$": ""}, regex = True)
school_student_summary_df.dtypes
school_student_summary_df["Per Student Budget"] = pd.to_numeric(school_student_summary_df["Per Student Budget"])
school_student_summary_df.dtypes
school_student_summary_df["Spending Ranges"] = pd.cut(school_student_summary_df["Per Student Budget"], spending_bins, labels=group_names)
spending_bin_df = school_student_summary_df.groupby("Spending Ranges").agg({"Average Math Score": "mean", "Average Reading Score": "mean", "% Passing Math":"mean", 
                                                                            "% Passing Reading": "mean", "% Overall Passing Rate": "mean"})
spending_bin_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


In [13]:
# scores by school size
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_student_summary_df["school_size"] = pd.cut(school_student_summary_df["Total Students"], size_bins, labels = group_names)
school_student_summary_df
school_size_bins = school_student_summary_df.groupby("school_size").agg({"Average Math Score": "mean", "Average Reading Score": "mean", "% Passing Math":"mean", 
                                                                            "% Passing Reading": "mean", "% Overall Passing Rate": "mean"})
school_size_bins

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


In [14]:
# scores by school type
school_type_df = pd.merge(school_df, student_df, how = "inner", on="school_name")
school_type_df = school_type_df.groupby("type").agg({"Student ID": "count", "math_score": ["mean", high_math_score_stu],
                                                 "reading_score": ["mean", high_reading_score_stu]})
school_type_df.columns = school_type_df.columns.map("_".join)
school_type_df = school_type_df.rename(columns={"Student ID_count": "Total Student", "math_score_mean": "Average Math Score",
                                             "math_score_high_math_score_stu": "Math Highscore Students", 
                                              "reading_score_mean" : "Average Reading Score",
                                              "reading_score_high_reading_score_stu":"Reading Highscore Students"})
school_type_df["% Passing Math"] = (school_type_df["Math Highscore Students"]/school_type_df["Total Student"]) * 100
school_type_df["% Passsing Reading"] = (school_type_df["Reading Highscore Students"]/school_type_df["Total Student"]) * 100
school_type_df["% Overall Passing Rate"] = (school_type_df["% Passing Math"]+school_type_df["% Passsing Reading"])/2
school_type_df.drop(["Total Student", "Math Highscore Students", "Reading Highscore Students"], axis = 1, inplace= True)
school_type_df.index.rename("School Type", inplace=True)
school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passsing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
